In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy, scipy as sklearn, librosa, urllib
import librosa.display
from IPython.display import Audio
import json 
import seaborn as sns
from sklearn.cluster import KMeans
import csv
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, recall_score, precision_score, f1_score
import keras
from sklearn.decomposition import PCA

from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler

from itertools import cycle
from sklearn.manifold import TSNE
from sklearn.metrics import roc_curve, auc, silhouette_score,roc_auc_score, precision_recall_fscore_support
from tqdm import tqdm

import intel_npu_acceleration_library


c:\Users\anton\Documents\PhD\Speech_Emotion_Recognition\EMOVO_dataset\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def setLabel(i): 
    emotions = ["neutrality", "calm", "joy", "sadness", "anger", "fear", "disgust", "surprise"] #calm è extra
    return emotions[i]

with open("RAVDESS/data.csv", 'r+') as f:
    f.truncate(0)

main_dir = "RAVDESS"
with open('RAVDESS/data.csv', 'w', newline='') as csvfile:
    fieldnames = ['file_name', 'label','actor']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for index,dir in enumerate(os.listdir(main_dir)):
        actual = os.path.join(main_dir,dir)
        if dir.endswith(".csv") or dir.endswith(".pkl") :
            continue
        if int(dir.split("_")[-1]) % 2 == 0:
            actor = "f" + str((index//2) +1)    
        else:
            actor = "m" + str((index//2) +1)
        for audio in os.listdir(actual):
            emo = int(audio.split("-")[2]) 
            writer.writerow({'file_name':os.path.join(dir,audio),'label':setLabel(emo-1),'actor':actor})

In [4]:
data_df = pd.read_csv("RAVDESS/data.csv")
data_df.drop(data_df[data_df['label'] == "calm"].index, inplace = True)
data_df = data_df.reset_index()

In [8]:
def get_max_min(files):
    min_, max_ = 100, 0
    for file in files:
        sound_file, samplerate = librosa.load(file)
        t = sound_file.shape[0] / samplerate
        if t < min_:
            min_ = t
        if t > max_:
            max_ = t

    return max_, min_

In [9]:
def extract_new(file,pad):
    X, sample_rate = librosa.load(file)
    max_ = X.shape[0] / sample_rate
    if max_ < pad:
        length = (pad * sample_rate) - X.shape[0]
        X = np.pad(X, (0, int(length)), 'constant')
    
    stft = np.abs(librosa.stft(X, n_fft=662, hop_length=221))
    # result = np.array([])
    result = []

    # mfccs = np.mean(.T, axis=0)

    mfccs = librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=13, n_fft=662, hop_length=221)

    # result = np.hstack((result, mfccs))
    result.append((mfccs))

    # chroma = np.mean(.T, axis=0)
    chroma = librosa.feature.chroma_stft(S=stft, sr=sample_rate, n_fft=662, hop_length=221)
    # result = np.hstack((result, chroma))
    result.append(chroma)

    # mel = np.mean(.T, axis=0) 
    mel = librosa.feature.melspectrogram(y=X, sr=sample_rate, n_fft=662, hop_length=221)
    # result = np.hstack((result, mel))
    result.append(mel)
    
    # contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T, axis=0)
    contrast = librosa.feature.spectral_contrast(S=stft, sr=sample_rate, n_fft=662, hop_length=221)
    # result = np.hstack((result, contrast))
    result.append(contrast)

    # tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
    # result = np.hstack((result, tonnetz))
    return result

In [10]:
max, min = get_max_min('RAVDESS/'+data_df.file_name)
output_length = int(np.ceil(max * (22050) / (221)))
train_data = pd.DataFrame(columns=['filename', 'features', 'label'])

features = []
for index, file in zip(data_df.index, data_df.file_name):
    train_data.loc[index] = [file, extract_new('RAVDESS/'+file, max), data_df.label[index]]

In [32]:
X1 = np.empty((0, output_length))
X2 = np.empty((0, output_length))
X3 = np.empty((0, output_length))
X4 = np.empty((0, output_length))

for data in tqdm(train_data["features"]):
    X1 = np.vstack((X1, data[0]))
    X2 = np.vstack((X2, data[1]))
    X3 = np.vstack((X3, data[2]))
    X4 = np.vstack((X4, data[3]))

X1 = X1.reshape(len(data_df),output_length,-1)
X2 = X2.reshape(len(data_df),output_length,-1)
X3 = X3.reshape(len(data_df),output_length,-1)
X4 = X4.reshape(len(data_df),output_length,-1)

100%|██████████| 1248/1248 [01:48<00:00, 11.54it/s]


In [64]:
data_classes = (list((train_data["label"].unique())))
Y = keras.utils.to_categorical(list((train_data["label"].apply(data_classes.index))))
# X = np.stack(train_data["features"]) 
X = np.expand_dims(np.concatenate([X1,X2,X3,X4], axis=2),3)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=22)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=22)
X_train.shape

(898, 527, 160, 1)

In [35]:
X1_n = np.expand_dims(X1,axis=3)
X_train, X_test, y_train, y_test = train_test_split(X1_n, Y, test_size=0.1, random_state=22)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=22)
X_train.shape

(898, 527, 13, 1)

In [65]:
def get_cnn(input_shape):
    model = keras.Sequential()

    model.add(keras.layers.Input(shape=input_shape))

    model.add(keras.layers.Conv2D(256, 3, activation='relu' ))
    model.add(keras.layers.MaxPooling2D(padding='same'))
    model.add(keras.layers.Dropout(rate=0.3))

    model.add(keras.layers.Conv2D(128, 3, activation='relu'))
    model.add(keras.layers.MaxPooling2D(padding='same'))
    model.add(keras.layers.Dropout(rate=0.3))


    model.add(keras.layers.Conv2D(64, 3, activation='relu'))
    model.add(keras.layers.MaxPooling2D(padding='same'))
    model.add(keras.layers.Dropout(rate=0.3))

    model.add(keras.layers.GlobalAveragePooling2D())
    model.add(keras.layers.Dense(1024, activation='relu'))
    
    model.add(keras.layers.Dense(256, activation='relu'))
    model.add(keras.layers.Dense(64, activation='relu'))

    model.add(keras.layers.Dense(7, activation='softmax'))

    optimzer = keras.optimizers.Adam()
    model.compile(loss='categorical_crossentropy', optimizer=optimzer, metrics=['accuracy'])

    return model

In [66]:
from datetime import datetime  
name = datetime.now().strftime("model/SER_RAVDESS_2d_no_rocket_%d_%m_%Y_%H_%M_%S.keras")  

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath = name,
        save_best_only=True,
        verbose=1,
        monitor="val_loss"),

    keras.callbacks.EarlyStopping(  
        monitor="val_loss",
        min_delta=0.001,
        patience=10,
        verbose=1,
        mode="auto",
        restore_best_weights=True
    )
]

model = get_cnn((X_train.shape[1:]))
model.summary()

Model: "sequential_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_29 (Conv2D)              │ (None, 525, 158, 256)  │         2,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_22 (MaxPooling2D) │ (None, 263, 79, 256)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_22 (Dropout)            │ (None, 263, 79, 256)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_30 (Conv2D)              │ (None, 261, 77, 128)   │       295,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_23 (MaxPooling2D) │ (None, 131, 39, 128)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_23 (Dropout)            │ (None, 131, 39, 128)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_31 (Conv2D)              │ (None, 129, 37, 64)    │        73,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_24 (MaxPooling2D) │ (None, 65, 19, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_24 (Dropout)            │ (None, 65, 19, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_8      │ (None, 64)             │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_32 (Dense)                │ (None, 1024)           │        66,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_33 (Dense)                │ (None, 256)            │       262,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_34 (Dense)                │ (None, 64)             │        16,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_35 (Dense)                │ (None, 7)              │           455 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 717,255 (2.74 MB)

 Trainable params: 717,255 (2.74 MB)

 Non-trainable params: 0 (0.00 B)

In [67]:
history = model.fit(X_train, y_train, 
                       validation_data=(X_val,y_val), 
                       batch_size=32,
                       epochs=1000,
                       callbacks=callbacks)


print(f"Loss : {model.evaluate(X_test,y_test)[0]}, Accuracy : {model.evaluate(X_test,y_test)[1]}")

Epoch 1/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.1448 - loss: 1.9729
Epoch 1: val_loss improved from inf to 1.92705, saving model to model/SER_RAVDESS_2d_no_rocket_04_10_2024_10_52_42.keras
29/29 ━━━━━━━━━━━━━━━━━━━━ 136s 5s/step - accuracy: 0.1448 - loss: 1.9723 - val_accuracy: 0.1378 - val_loss: 1.9270
Epoch 2/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.1683 - loss: 1.9191
Epoch 2: val_loss did not improve from 1.92705
29/29 ━━━━━━━━━━━━━━━━━━━━ 157s 5s/step - accuracy: 0.1681 - loss: 1.9195 - val_accuracy: 0.1422 - val_loss: 1.9298
Epoch 3/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.1747 - loss: 1.9312
Epoch 3: val_loss improved from 1.92705 to 1.88765, saving model to model/SER_RAVDESS_2d_no_rocket_04_10_2024_10_52_42.keras
29/29 ━━━━━━━━━━━━━━━━━━━━ 148s 5s/step - accuracy: 0.1741 - loss: 1.9311 - val_accuracy: 0.1778 - val_loss: 1.8876
Epoch 4/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.1699 - loss: 1.9028
Epoch 4: val_loss i

In [73]:
data_ita_df = pd.read_csv("EMOVO_dataset/data.csv")

max_ita, min = get_max_min('EMOVO_dataset/'+data_ita_df.file_name)
output_length_ita = int(np.ceil(max_ita * (22050) / (512)))
train_ita_data = pd.DataFrame(columns=['filename', 'features', 'label'])

features = []
for index, file in zip(data_ita_df.index, data_ita_df.file_name):
    train_ita_data.loc[index] = [file, extract_new('EMOVO_dataset/'+file, max_ita), data_ita_df.label[index]]


X1_ita = np.empty((0, output_length_ita))
X2_ita = np.empty((0, output_length_ita))
X3_ita = np.empty((0, output_length_ita))
X4_ita = np.empty((0, output_length_ita))

for data in tqdm(train_ita_data["features"]):
    X1_ita = np.vstack((X1_ita, data[0]))
    X2_ita = np.vstack((X2_ita, data[1]))
    X3_ita = np.vstack((X3_ita, data[2]))
    X4_ita = np.vstack((X4_ita, data[3]))

X1_ita = X1_ita.reshape(len(data_ita_df),output_length_ita,-1)
X2_ita = X2_ita.reshape(len(data_ita_df),output_length_ita,-1)
X3_ita = X3_ita.reshape(len(data_ita_df),output_length_ita,-1)
X4_ita = X4_ita.reshape(len(data_ita_df),output_length_ita,-1)

100%|██████████| 588/588 [00:36<00:00, 16.13it/s]


In [ ]:
data_classes = (list((train_ita_data["label"].unique())))
Y = keras.utils.to_categorical(list((train_ita_data["label"].apply(data_classes.index))))
# X = np.stack(train_ita_data["features"]) 
X = np.expand_dims(np.concatenate([X1_ita,X2_ita,X3_ita,X4_ita], axis=2),3)

X_train, X_val,Yin, y_test = train_test_split(X, Y, test_size=0.1, random_state=22)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=22)


print(f"Loss : {model.evaluate(X,Y)[0]}, Accuracy : {model.evaluate(X,Y)[1]}")

In [69]:
from keras import layers, models
def get_model(input_shape):
    inputs = layers.Input(shape=input_shape)
    encoder = layers.LSTM(128)(inputs)
    drop = layers.Dropout(0.3)(encoder)
    hidden = layers.Dense(32, activation='relu')(drop)
    outputs = layers.Dense(7, activation='softmax')(hidden)
    
    model = models.Model(inputs, outputs)
    optimizer = keras.optimizers.Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=["accuracy"])

    return model

In [70]:
X_train_sq = np.squeeze(X_train)
X_val_sq = np.squeeze(X_val)
X_test_sq = np.squeeze(X_test)

In [71]:
X_train_sq.shape

(898, 228, 197)

In [72]:
LSTM_model = get_model((X_train_sq.shape[1:]))
LSTM_model.summary()


from datetime import datetime  
name = datetime.now().strftime("ser_lstm_%d_%m_%Y_%H_%M_%S.keras")  

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath = name,
        save_best_only=True,
        verbose=1,
        monitor="val_loss"),

    keras.callbacks.EarlyStopping(  
        monitor="val_loss",
        min_delta=0.001,
        patience=20,
        verbose=1,
        mode="auto",
        restore_best_weights=True
    )
]


LSTM_history = LSTM_model.fit(X_train_sq, y_train, 
                       validation_data=(X_val_sq,y_val), 
                       batch_size=32,
                       epochs=1000,
                       verbose=1,
                       callbacks=callbacks)


print(f"Loss : {LSTM_model.evaluate(X_test_sq,y_test)[0]}, Accuracy : {LSTM_model.evaluate(X_test_sq,y_test)[1]}")

Model: "functional_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_8 (InputLayer)      │ (None, 228, 197)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_7 (LSTM)                   │ (None, 128)            │       166,912 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 7)              │           231 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 171,271 (669.03 KB)

 Trainable params: 171,271 (669.03 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/1000
28/29 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.1429 - loss: 2.1439
Epoch 1: val_loss improved from inf to 1.92317, saving model to ser_lstm_02_10_2024_14_48_25.keras
29/29 ━━━━━━━━━━━━━━━━━━━━ 3s 68ms/step - accuracy: 0.1432 - loss: 2.1364 - val_accuracy: 0.1333 - val_loss: 1.9232
Epoch 2/1000
28/29 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.1655 - loss: 1.9373
Epoch 2: val_loss improved from 1.92317 to 1.91395, saving model to ser_lstm_02_10_2024_14_48_25.keras
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 64ms/step - accuracy: 0.1656 - loss: 1.9372 - val_accuracy: 0.1600 - val_loss: 1.9140
Epoch 3/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.1984 - loss: 1.9079
Epoch 3: val_loss improved from 1.91395 to 1.89785, saving model to ser_lstm_02_10_2024_14_48_25.keras
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 60ms/step - accuracy: 0.1984 - loss: 1.9079 - val_accuracy: 0.1867 - val_loss: 1.8979
Epoch 4/1000
28/29 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.1966 - loss: 1.9157

In [92]:
def obtain_df(X,data_df):
    X_df = pd.DataFrame(columns=['filename', 'features', 'label'])
    X = X.reshape(X.shape[0],-1)


    for index, file in zip(data_df.index, data_df.file_name):
        X_df.loc[index] = [file, pd.Series(X[index]), data_df.label[index]]
    X = pd.DataFrame(X_df["features"])
    return X

from sktime.transformations.panel.rocket import Rocket

def get_rocket(X):
    trf = Rocket(num_kernels=512) 
    trf.fit(X) 
    X_ = trf.transform(X)
    return X_


In [91]:
X1.reshape(X1.shape[0],-1).shape

(1248, 11400)

In [96]:
X1_ = (get_rocket(obtain_df(X1,data_df))).to_numpy()
X2_ = get_rocket(obtain_df(X2,data_df)).to_numpy()
X3_ = get_rocket(obtain_df(X3,data_df)).to_numpy()
X4_ = get_rocket(obtain_df(X4,data_df)).to_numpy()

In [105]:
X_.shape

(1248, 4096)

In [106]:
data_classes = (list((train_data["label"].unique())))
Y = keras.utils.to_categorical(list((train_data["label"].apply(data_classes.index))))
# X = np.stack(train_data["features"])
X_ = np.hstack([X1_,X2_,X3_,X4_])
X_ = np.expand_dims(X_,2)

In [107]:
X_train_, X_test_, y_train_, y_test_ = train_test_split(X_, Y, test_size=0.1, random_state=22)
X_train_, X_val_, y_train_, y_val_ = train_test_split(X_train_, y_train_, test_size=0.2, random_state=22)


In [108]:
LSTM_model = get_model(X_train_.shape[1:])
LSTM_model.summary()


from datetime import datetime  
name = datetime.now().strftime("models/LSTM_RAVDESS_%d_%m_%Y_%H_%M_%S.keras")  

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath = name,
        save_best_only=True,
        verbose=1,
        monitor="val_loss"),

    keras.callbacks.EarlyStopping(  
        monitor="val_loss",
        min_delta=0.001,
        patience=20,
        verbose=1,
        mode="auto",
        restore_best_weights=True
    )
]


LSTM_history = LSTM_model.fit(X_train_, y_train_, 
                       validation_data=(X_val_,y_val_), 
                       batch_size=32,
                       epochs=1000,
                       verbose=1,
                       callbacks=callbacks)


print(f"Loss : {LSTM_model.evaluate(X_test_,y_test_)[0]}, Accuracy : {LSTM_model.evaluate(X_test_,y_test_)[1]}")

Model: "functional_17"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_11 (InputLayer)     │ (None, 4096, 1)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_9 (LSTM)                   │ (None, 128)            │        66,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 7)              │           231 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 70,919 (277.03 KB)

 Trainable params: 70,919 (277.03 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 972ms/step - accuracy: 0.1783 - loss: 1.9495
Epoch 1: val_loss improved from inf to 1.92714, saving model to models/LSTM_RAVDESS_02_10_2024_15_20_24.keras
29/29 ━━━━━━━━━━━━━━━━━━━━ 33s 1s/step - accuracy: 0.1777 - loss: 1.9496 - val_accuracy: 0.1422 - val_loss: 1.9271
Epoch 2/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 975ms/step - accuracy: 0.1522 - loss: 1.9330
Epoch 2: val_loss improved from 1.92714 to 1.92086, saving model to models/LSTM_RAVDESS_02_10_2024_15_20_24.keras
29/29 ━━━━━━━━━━━━━━━━━━━━ 32s 1s/step - accuracy: 0.1519 - loss: 1.9332 - val_accuracy: 0.1511 - val_loss: 1.9209
Epoch 3/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.1616 - loss: 1.9247
Epoch 3: val_loss improved from 1.92086 to 1.91947, saving model to models/LSTM_RAVDESS_02_10_2024_15_20_24.keras
29/29 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - accuracy: 0.1618 - loss: 1.9248 - val_accuracy: 0.1822 - val_loss: 1.9195
Epoch 4/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - acc

KeyboardInterrupt: 

In [85]:
keras.models.save_model(LSTM_model, "models/LSTM_no_rocket_029.keras")